**Collborative filtering**

In [66]:
import torch as torch

class CF_model:
    def __init__(self, data: torch.Tensor, d: int=5) -> None:
        self.m = data.shape[0]
        self.n = data.shape[1]
        self.d = d
        self.U = torch.rand(self.m, d, requires_grad=True) #m users
        self.V = torch.rand(self.n, d, requires_grad=True) #n items

    def fit(self, data: torch.Tensor) -> None:
        max_iter = 100
        lr = 0.01

        for i in range(1, max_iter+1):

            if self.U.grad is not None:
                self.U.grad.zero_()
            if self.V.grad is not None:
                self.V.grad.zero_()

            mask = (data != -1).float() # missing data labeled as -1
            reconstructed = torch.matmul(self.U, self.V.t())
            loss = torch.sum(((reconstructed - data) * mask) ** 2) / torch.sum(mask)
            loss.backward()

            with torch.no_grad():
                self.U -= lr * self.U.grad
                self.V -= lr * self.V.grad
        
        self.loss = loss

    def pred(self, uid: int) -> int:
        return torch.argmax(self.U[uid] @ self.V.t()).squeeze().item()

In [68]:
# Example usage
data = torch.tensor([[5, 3, -1], [4, -1, 5], [-1, 2, 3]], dtype=torch.float32)
model = CF_model(data)
model.fit(data)

uid = 2
print(f"rec for user {uid} is item {model.pred(uid)}")  # Predict the highest-rated item for user 0


print('--- training loss ---')
print(model.loss)

matrix = model.U @ model.V.T
print('--- user <> item matrix ---')
print(matrix)

rec for user 2 is item 2
--- training loss ---
tensor(0.0616, grad_fn=<DivBackward0>)
--- user <> item matrix ---
tensor([[4.6235, 3.0870, 5.0463],
        [4.3788, 2.5397, 4.7810],
        [2.5653, 2.0961, 2.8916]], grad_fn=<MmBackward0>)
